In [2]:
from fred import Fred
import numpy as np
import pandas as pd
from datetime import datetime
import time

# FRED API key
# key = 'enter your key'
%run -i fred_api_key

## Getting Data from FRED ##
fr = Fred(api_key = key, response_type = 'df')

# Setting data parameters
start = datetime(1990, 1, 1)
end = datetime(2019, 4, 1)

params = {'observation_start': start.strftime('%Y-%m-%d'),
          'observation_end': end.strftime('%Y-%m-%d')}

# Load list of data series
fseries = pd.read_csv("FRED_data_series.csv")

In [2]:
def quarterMean(series, dates):
    ret = []
    for start, end in zip(dates[:-1], dates[1:]):
        batch = series.loc[(series.date >= start) & (series.date < end), 'value']
        shrink = np.mean(batch)
        ret.append(shrink)
    return ret

def growthRate(series):
    return np.exp(np.diff(np.log(series))) - 1

class DataGetter:
    include = True
    def __init__(self, code, dates, method, params):
        self.code = code
        self.dates = dates
        self.method = method
        self.params = params
        self.start = fr.series.details(code).observation_start[0]

    def levelShrink(self, dates, series):
        ret = []
        for start, end in zip(dates[:-1], dates[1:]):
            batch = series.loc[(series.date >= start) & (series.date < end), 'value']
            if (batch.isnull().all()):
                self.include = False
                break
            if (all((x == 0) | np.isnan(x) for x in batch)):
                self.include = False
                break
            shrink = np.prod(1 + batch) - 1
            ret.append(shrink)
        return ret

    def percentShrink(self, dates, series):
        ret = []
        for start, end in zip(dates[:-1], dates[1:]):
            batch = series.loc[(series.date >= start) & (series.date < end), 'value']
            if (batch.isnull().all()):
                self.include = False
                break
            if (all((x == 0) | np.isnan(x) for x in batch)):
                self.include = False
                break
            shrink = np.sum(np.diff(batch))
            ret.append(shrink)
        return ret

    def getter(self):
        series = fr.series.observations(self.code, params = self.params)
        series.dropna(axis = 0, inplace = True)
        if(method):
            series = self.percentShrink(self.dates, series)
        else:
            series = pd.DataFrame({'date' : series.date[1:], 'value': growthRate(series.value)})
            series = self.levelShrink(self.dates, series)
        return series

In [4]:
# Loading GDP
gdp = fr.series.observations("GDPC1", params = params)
gdp.drop(labels = ['realtime_end', 'realtime_start'], axis = 1, inplace = True)
gdp.rename(columns = {'value' : 'gdp'}, inplace = True)

# Retrieving data from Fred
n = fseries.shape[0]
dates = gdp.date

data = pd.DataFrame({'date' : dates[1:]})

for i in range(n):
    code = fseries.loc[i, 'file']
    method = fseries.loc[i, 'method']
    
    dg = DataGetter(code, dates, method, params)
    series = dg.getter()
    
    if ((dg.start <= start) & dg.include):
        data[code] = series
    if (i % 100  == 0): 
        print(i)
    time.sleep(.5)

# adding in zero-indexed data
data['MEIM683SFRBCHI'] = quarterMean(fr.series.observations('MEIM683SFRBCHI', params = params), dates)
data['RMEIM683SFRBCHI'] = quarterMean(fr.series.observations('RMEIM683SFRBCHI', params = params), dates)

# adding lagged GDP
lag1 = {'observation_start': "1989-10-01",
        'observation_end': "2018-07-01"}

lag2 = {'observation_start': "1989-07-01",
        'observation_end': "2018-04-01"}

data['GDPLAG1'] = growthRate(fr.series.observations('GDPC1', params = lag1).value)
data['GDPLAG2'] = growthRate(fr.series.observations('GDPC1', params = lag2).value)

0
100
200
300


URLError: <urlopen error [WinError 10053] An established connection was aborted by the software in your host machine>

In [47]:
from sklearn import gaussian_process as gp
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.metrics import mean_squared_error as MSE

# Setting data parameters
start = datetime(1990, 1, 1)
end = datetime(2019, 4, 1)

params = {'observation_start': start.strftime('%Y-%m-%d'),
          'observation_end': end.strftime('%Y-%m-%d')}

fr = Fred(api_key = key, response_type = 'df')
gdp = fr.series.observations("GDPC1", params = params)

def growthRate(series):
    return np.exp(np.diff(np.log(series))) - 1

# fitting regression
X = pd.read_csv("data.csv").drop('date', axis = 1)
y = growthRate(gdp.value)

# Setting the list of potential kernels 
ker_rbf = gp.kernels.ConstantKernel() + gp.kernels.RBF() + gp.kernels.WhiteKernel()
# ker_matern = gp.kernels.ConstantKernel() + gp.kernels.Matern() + gp.kernels.WhiteKernel()
# ker_rq = gp.kernels.ConstantKernel() + gp.kernels.RationalQuadratic() + gp.kernels.WhiteKernel()

gp_model = gp.GaussianProcessRegressor(kernel = kernel)
gp_model.fit(X,y)

# Preallocating memory for LOO predictions
gp_errors = pd.DataFrame({'target': y, 
                          'mu': np.zeros(len(y)), 
                          'sigma': np.zeros(len(y))})

C:\Users\Davis Berlind\Miniconda3\lib\site-packages\fred\helpers\__init__.py:36: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  frame = frame.convert_objects(convert_numeric=True)


0.00316**2 + RBF(length_scale=1) + WhiteKernel(noise_level=1e-05)
7.158672362389217e-15
0.00316**2 + Matern(length_scale=1, nu=1.5) + WhiteKernel(noise_level=1e-05)
7.158672324154794e-15
0.00552**2 + RationalQuadratic(alpha=0.000118, length_scale=4.11e+03) + WhiteKernel(noise_level=1.71e-05)
1.2576875622614324e-05


In [15]:
from statsmodels.tsa.stattools import adfuller
X = pd.read_csv("data.csv").drop('date', axis = 1)

# p-value <= 0.05: Reject the null hypothesis (H0) 
# the data does not have a unit root and is stationary.

adfuller_results = X.apply(adfuller, axis = 1)
failed = 0
p_val = 0.05
for i, result in zip(range(X.shape[1]), adfuller_results):
    if (result[1] > p_val): 
        failed += 1
        print(i)
print(failed)
X.columns[48]

49
1


'CAPUTLG314S'

In [40]:
gp_model = gp.GaussianProcessRegressor(kernel = ker_matern)
gp_model.fit(X, y)
gp_model.predict(X)


array([ 0.00362905,  0.00066522, -0.0091037 , -0.00467839,  0.00779711,
        0.00505292,  0.00348495,  0.0119704 ,  0.01084343,  0.00988262,
        0.01043154,  0.00167399,  0.00582414,  0.0047731 ,  0.013601  ,
        0.00970207,  0.01355032,  0.00584509,  0.01145513,  0.00354753,
        0.00298338,  0.00850669,  0.00679014,  0.00748939,  0.01667831,
        0.00896982,  0.01038372,  0.00645558,  0.01661985,  0.01250836,
        0.00858808,  0.0099947 ,  0.00925798,  0.01252555,  0.01615864,
        0.00946462,  0.00769151,  0.01309652,  0.01699568,  0.00361757,
        0.01831146,  0.00133554,  0.00622883, -0.00285175,  0.0058455 ,
       -0.00415034,  0.00272369,  0.00874505,  0.00605726,  0.00444567,
        0.00154821,  0.00554761,  0.00860251,  0.01698192,  0.01148156,
        0.00533742,  0.00762069,  0.00945319,  0.01001664,  0.01106952,
        0.00461557,  0.00891325,  0.00631457,  0.01330274,  0.00233697,
        0.00154631,  0.00851857,  0.00235491,  0.00572867,  0.00